## Problem Statement



Define scores for referrals who have been giving fruitful refers. Fruitful refers translate to the performance of the referees who joined by referral. If a client has a higher trade activation ratio for his/her refers, his/her score must be higher.




## Features



* Age
* income_per_annum
* occupation
* Z20
* app_status
* E2 channel
* iskarvy
* KRA

========= trade behaviour of referrer =========
* number of logins in last year
* num_trades in last year
* months since last login
* months since last trade

========= trade behaviour of referees =========
* average num_logins per month post account open
* average num_trades per month post account open 

## importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from datetime import datetime
from datetime import date

In [3]:
df_leads = pd.read_csv("../../../data/CRM_leads_zoho.csv")

df_client =  pd.read_csv("../../../data/dw_mst_client.csv")
df_client = df_client[df_client["data_source"] == "Equity"]


## Data Preperation - referrers(trarget =  1)

In [87]:
# df_leads["Reference_Number"] = df_leads["Reference_Number"].astype(str).str.split(".").str[0] 

# df_leads["Reference_Number"] = df_leads["Reference_Number"].astype(str).str.split(".").str[0] 

df_referrer = pd.concat([df_leads[~df_leads["Reference_Number"].isna()].groupby(by = ["Reference_Number"]).nunique()[["Mobile"]], 
                        df_leads[(~df_leads["Reference_Number"].isna()) & (df_leads["Lead_Status"] == "Product Sold")].groupby(by = ["Reference_Number"]).nunique()[["Mobile"]]], 
                        axis = 1)

df_referrer = df_referrer.reset_index()

df_referrer = df_referrer[df_referrer["Reference_Number"].str.len() == 10]

df_referrer.columns = ["Reference_Number", "refer_count", "activation_count"]

df_referrer["activation_count"] = df_referrer["activation_count"].fillna(0)

df_referrer


,Reference_Number,refer_count,activation_count
10,6000015546,1,0.0
11,6000016229,1,0.0
12,6000027031,1,0.0
13,6000031654,31,23.0
14,6000037524,3,0.0
...,...,...,...
34530,9999888891,1,0.0
34531,9999893553,1,1.0
34532,9999903825,2,2.0
34533,9999920530,1,1.0


#### adding demographcs to referrers

In [88]:
df_referrer.rename(columns = {"Mobile": "refer_count"}, inplace = True)
df_client["cm_mobile"] = df_client["cm_mobile"].astype(str).str.split(".").str[0]
df_referrer = pd.merge(df_referrer, df_client, 
                      left_on = "Reference_Number", 
                      right_on = "cm_mobile", how = "inner")[["cm_cd", "cm_mobile", "refer_count", "activation_count", 
                                                             "cm_dob", "IncomePA", "cm_occup", "E2Channel", "IsKarvy", 
                                                             "KRA YN", "Last_MobileLoginDate", "LTD"]] 

df_model = df_referrer.copy()


## adding age 
now = datetime.now()
df_model["cm_dob"] = df_model["cm_dob"].astype(str).str.split(".").str[0]
df_model["cm_dob"] = pd.to_datetime(df_model["cm_dob"], errors = "coerce")
df_model["age"] =  (now -df_model["cm_dob"]).astype('<m8[Y]')
del df_model["cm_dob"]
print("age column added!!")


# adding months since last login
df_model["Last_MobileLoginDate"] = df_model["Last_MobileLoginDate"].astype(str).str.split(".").str[0]
df_model["Last_MobileLoginDate"] = pd.to_datetime(df_model["Last_MobileLoginDate"], errors = "coerce")
df_model["months_since_last_login"] =  (now -df_model["Last_MobileLoginDate"])/np.timedelta64(1, 'M')
del df_model["Last_MobileLoginDate"]
df_model["months_since_last_login"] = df_model["months_since_last_login"].fillna(132)
print("month since last login added")

# adding months since last trade 
df_model["LTD"] = df_model["LTD"].astype(str).str.split(".").str[0]
df_model["LTD"] = pd.to_datetime(df_model["LTD"], errors = "coerce")
df_model["months_since_last_trade"] =  (now -df_model["LTD"])/np.timedelta64(1, 'M')
del df_model["LTD"]
df_model["months_since_last_trade"] = df_model["months_since_last_trade"].fillna(132)
print("month since last trade added")


df_thv = pd.read_csv("../../../data/THV_all_clients_20211126.csv")
df_model = pd.merge(df_model, 
                   df_thv, 
                   left_on = "cm_cd", right_on = "client_code", 
                   how = "left")[list(df_model.columns) +  ["dp_holding"]]
df_model["dp_holding"].fillna(0)
print("dp holdng added!!")



## adding trades and logins in one year

df_logins_one_year = pd.read_csv("../../../data/logins_in_one_year.csv") 
df_model.rename(columns = {"ClientCode": "client_code"}, inplace = True)
df_model = pd.merge(df_model, df_logins_one_year, 
                   left_on = "cm_cd", 
                   right_on = "ClientCode", how = "left")[list(df_model.columns) + ["total_logins_one_year"]]
df_model["total_logins_one_year"] = df_model["total_logins_one_year"].fillna(0)

df_trades_one_year = pd.read_csv("../../../data/trades_in_one_year.csv") 
df_model = pd.merge(df_model, df_trades_one_year, 
                   left_on = "cm_cd", 
                   right_on = "Client_Code", how = "left")[list(df_model.columns) + ["total_trades_one_year"]]
df_model["total_trades_one_year"] = df_model["total_trades_one_year"].fillna(0)
print("added trades and logins in one year!!")

age column added!!
month since last login added
month since last trade added
dp holdng added!!
added trades and logins in one year!!


In [89]:
df_referrer = df_model.copy()

In [90]:
df_referrer

,cm_cd,cm_mobile,refer_count,activation_count,IncomePA,cm_occup,E2Channel,IsKarvy,KRA YN,age,months_since_last_login,months_since_last_trade,dp_holding,total_logins_one_year,total_trades_one_year
0,76556800,6000015546,1,0.0,5_10L,P,CAT,0,N,22.0,1.993668,132.000000,0.0,10.0,0.0
1,49282624,6000016229,1,0.0,<=1L,S,CAT,0,Y,22.0,1.040877,1.730829,0.0,35.0,1.0
2,60000270,6000027031,1,0.0,<=1L,P,CAT,0,N,22.0,1.336571,1.270861,0.0,4.0,1.0
3,64606053,6000031654,31,23.0,<=1L,S,CAT,0,N,32.0,0.515198,3.373573,0.0,208.0,5.0
4,86330038,6000037524,3,0.0,<=1L,S,CAT,0,Y,23.0,1.238006,2.847895,0.0,69.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33381,99998155,9999815503,1,0.0,<=1L,B,CAT,0,N,27.0,132.000000,132.000000,0.0,0.0,0.0
33382,INDERSI5,9999888891,1,0.0,1_5L,NaN,Premia,0,N,36.0,0.515198,0.745183,102677.0,1367.0,292.0
33383,59684125,9999893553,1,1.0,<=1L,P,CAT,0,N,25.0,0.515198,0.745183,0.0,1635.0,63.0
33384,SAMPATIW,9999903825,2,2.0,5_10L,P,CAT,0,N,33.0,0.515198,1.533700,0.0,381.0,41.0


In [91]:
df_referrer.to_csv("../inter_data/referrers.csv")

## Data preperation - not referred(target = 0)

In [92]:

# clients accquired in jan to mar 2021, traded not referred
df1 = df_client[(df_client["cm_opendt"] >= 2021010) & 
               (df_client["cm_opendt"] <= 20210331) & 
               (~df_client["FTD"].isna()) &
               ~(df_client["cm_cd"].isin(df_referrer["cm_cd"]))].iloc[:10000, :][["cm_cd"]]


# clients accquired in jan to mar 2021, not traded not referred
df2 = df_client[(df_client["cm_opendt"] >= 2021010) & 
               (df_client["cm_opendt"] <= 20210331) & 
               (df_client["FTD"].isna()) &
               ~(df_client["cm_cd"].isin(df_referrer["cm_cd"])) &
               ~((df_client["cm_cd"]).isin(df1["cm_cd"]))].iloc[:10000, :][["cm_cd"]]



# # # clients accquired in jan to mar 2021, trades < 3 and not referred
# # df_THV = pd.read_csv("../inter_data/THV_all_clients_20211126.csv")
# # df3 = df_client[(df_client["cm_opendt"] >= 2021010) & 
# #                (df_client["cm_opendt"] <= 20210331) & 
# #                (df_client["FTD"].isna()) &
# #                ~(df_client["cm_cd"].isin(df_referrer["cm_cd"])) &
# #                ~((df_client["cm_cd"]).isin(df2["cm_cd"])].iloc[:10000, :][["cm_cd"]]



# clients less than 10k dp holding
df_THV = pd.read_csv("../../../data/THV_all_clients_20211126.csv")
df4 = df_THV[(df_THV["dp_holding"] < 10000)][["client_code"]].iloc[:10000, :]
df4.rename(columns = {"client_code": "cm_cd"}, inplace =True)

                                                                                 


In [93]:
df_non_referrer = pd.concat([df1, df2, df4], axis = 0)

In [94]:
df_non_referrer.to_csv("../inter_data/referrers.csv")

#### Adding features to non_referrers

In [95]:
df_non_referrer["refer_count"] = [0]*len(df_non_referrer)
df_non_referrer["activation_count"] = [0]*len(df_non_referrer)


df_non_referrer.rename(columns = {
    "cm_cd": "client_code"
})
df_non_referrer = pd.merge(df_non_referrer.rename(columns = {"cm_cd": "client_code"}), 
                           df_client, 
                           left_on = "client_code", 
                           right_on = "cm_cd", how = "inner")[["cm_cd", "cm_mobile", "refer_count", "activation_count", 
                                                             "cm_dob", "IncomePA", "cm_occup", "E2Channel", "IsKarvy", 
                                                             "KRA YN", "Last_MobileLoginDate", "LTD"]] 

df_model = df_non_referrer.copy()


## adding age 
now = datetime.now()
df_model["cm_dob"] = df_model["cm_dob"].astype(str).str.split(".").str[0]
df_model["cm_dob"] = pd.to_datetime(df_model["cm_dob"], errors = "coerce")
df_model["age"] =  (now -df_model["cm_dob"]).astype('<m8[Y]')
del df_model["cm_dob"]
print("age column added!!")


# adding months since last login
df_model["Last_MobileLoginDate"] = df_model["Last_MobileLoginDate"].astype(str).str.split(".").str[0]
df_model["Last_MobileLoginDate"] = pd.to_datetime(df_model["Last_MobileLoginDate"], errors = "coerce")
df_model["months_since_last_login"] =  (now -df_model["Last_MobileLoginDate"])/np.timedelta64(1, 'M')
del df_model["Last_MobileLoginDate"]
df_model["months_since_last_login"] = df_model["months_since_last_login"].fillna(132)
print("month since last login added")

# adding months since last trade 
df_model["LTD"] = df_model["LTD"].astype(str).str.split(".").str[0]
df_model["LTD"] = pd.to_datetime(df_model["LTD"], errors = "coerce")
df_model["months_since_last_trade"] =  (now -df_model["LTD"])/np.timedelta64(1, 'M')
del df_model["LTD"]
df_model["months_since_last_trade"] = df_model["months_since_last_trade"].fillna(132)
print("month since last trade added")


df_thv = pd.read_csv("../../../data/THV_all_clients_20211126.csv")
df_model = pd.merge(df_model, 
                   df_thv, 
                   left_on = "cm_cd", right_on = "client_code", 
                   how = "left")[list(df_model.columns) +  ["dp_holding"]]
df_model["dp_holding"].fillna(0)
print("dp holding addded!!")


## adding trades and logins in one year

df_logins_one_year = pd.read_csv("../../../data/logins_in_one_year.csv") 
df_model.rename(columns = {"ClientCode": "client_code"}, inplace = True)
df_model = pd.merge(df_model, df_logins_one_year, 
                   left_on = "cm_cd", 
                   right_on = "ClientCode", how = "left")[list(df_model.columns) + ["total_logins_one_year"]]
df_model["total_logins_one_year"] = df_model["total_logins_one_year"].fillna(0)

df_trades_one_year = pd.read_csv("../../../data/trades_in_one_year.csv") 
df_model = pd.merge(df_model, df_trades_one_year, 
                   left_on = "cm_cd", 
                   right_on = "Client_Code", how = "left")[list(df_model.columns) + ["total_trades_one_year"]]
df_model["total_trades_one_year"] = df_model["total_trades_one_year"].fillna(0)
print("added trades and logins in one year!!")
df_non_referrer =  df_model.copy()

age column added!!
month since last login added
month since last trade added
dp holding addded!!
added trades and logins in one year!!


In [96]:
df_referrer.to_csv("../inter_data/referrer.csv")
df_non_referrer.to_csv("../inter_data/non_referrer.csv")

In [97]:
!jt -t onedork -T -N -kl 

